In [ ]:
from __future__ import print_function
import os, sys, StringIO, time, threading, collections
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)

import keras
import KerasTools as KT
import numpy as np

import ipywidgets
import skimage
import matplotlib.pyplot as plt

In [ ]:
stopEvent = threading.Event()
currentGame = collections.deque([], 1)
class GamePlot(threading.Thread):
    def __init__(self):
        threading.Thread.__init__(self, name="GamePlot")
        self.imbuf = StringIO.StringIO()
        self.img = ipywidgets.Image(width=256, height=256)
        display(self.img)
        self.plot_frame(np.zeros((10,10), np.uint8))
    def run(self):
        while not stopEvent.isSet():
            try: game = currentGame.pop()
            except IndexError: time.sleep(0.1)
            else:
                for frame in game:
                    self.plot_frame(frame)
                    time.sleep(0.1)
                time.sleep(0.5)
    def plot_frame(self, frame):
        f = skimage.transform.resize(frame,(256,256), order=0, mode='constant', anti_aliasing=False)
        plt.imsave(self.imbuf, f, vmin=0.0, vmax=1.0, cmap='nipy_spectral')
        self.imbuf.seek(0)           
        self.img.value = self.imbuf.buf
        
class GameStore:
    def __init__(self):
        self.plotter = GamePlot()
        self.plotter.start()
    def game_start(self, frame):
        self.gamestore = [frame]
    def game_frame(self, frame):
        self.gamestore.append(frame)
    def game_over(self):
        currentGame.append(self.gamestore)

In [ ]:
grid_size = 10
hidden_size = 100
nb_frames = 10

model = keras.models.Sequential()
model.add(keras.layers.Flatten(input_shape=(nb_frames, grid_size, grid_size)))
model.add(keras.layers.Dense(100, activation='relu'))
model.add(keras.layers.Dense(3))
model.compile(keras.optimizers.rmsprop(), "logcosh")
model.summary()

game = KT.qlearn.catch.Catch(grid_size)
agent = KT.qlearn.agent.Agent(model=model, memory_size=20000, nb_frames = nb_frames)
stopEvent.clear()
agent.train(game, batch_size=256, epochs=10, train_interval=32,
            epsilon=[0.5, 0.01], epsilon_rate=0.5, reset_memory=False,
            callbacks=[GameStore()])

stopEvent.set()

In [ ]:
grid_size = 10
hidden_size = 100
nb_frames = 10

model = keras.models.Sequential()
model.add(keras.layers.Reshape((nb_frames, grid_size*grid_size), input_shape=(nb_frames, grid_size, grid_size)))
model.add(keras.layers.GRU(10))
model.add(keras.layers.Dense(3))
model.compile(keras.optimizers.rmsprop(), "logcosh")
model.summary()

game = KT.qlearn.catch.Catch(grid_size)
agent = KT.qlearn.agent.Agent(model=model, memory_size=20000, nb_frames = nb_frames)
stopEvent.clear()
agent.train(game, batch_size=256, epochs=10, train_interval=32,
            epsilon=[0.5, 0.01], epsilon_rate=0.5, reset_memory=False,
            callbacks=[GameStore()])

stopEvent.set()